In [1]:
from pathlib import Path
import math
import os
import numpy as np
import pandas as pd
import time
from numba import jit

def extract_data():
    raiz = Path.cwd() #capturo diretório atual
    pasta_dados = Path.joinpath(raiz,"dados") #defino onde estão os problemas
    prob_por_arquivo = 10 #informando quantos problemas por arquivo eu tenho
    conjuntos = [10,20,50,100,200,500,1000]

    for conjunto in conjuntos:  #circulo nos arquivos
        dados[conjunto] = {}
        arquivo = f"sch"+str(conjunto)+'.txt'
        print("Iniciando problema {}".format(arquivo))
        caminho_problema = Path.joinpath(pasta_dados,arquivo)
        df = pd.read_fwf(caminho_problema) #importo no df
        df.columns=["n","p","a","b"]
        df.drop(columns=["n"], inplace=True) #jogo fora columa inútil
        n=df.iloc[0,0] #guardo o tamanho de itens do problema
        print("n = ", n)  
        df.dropna(inplace=True) #jogo fora linhas inúteis
        for problema_numero in range(prob_por_arquivo): #vou circular nos problemas dentro do arquivo. Cada iteração deste for é um problema.
            prob = df[n*problema_numero:n*(problema_numero+1)]
            dados[conjunto][problema_numero+1] = {
                'pi': prob["p"].to_numpy(),
                'ai': prob["a"].to_numpy(),
                'bi': prob["b"].to_numpy()
            }
            ## nesse ponto tenho o parâmetros do problema atual guardadas nas variáveis pi, ai e bi, prontos para serem utilizados na modelagem.
    return dados

@jit(nopython=True)
def transforma(solucao):
    set_E = np.nonzero(solucao == True)[0] #[0] para retornar o np array e não o tuple.
    set_T = np.nonzero(solucao == False)[0]
    return set_E,set_T

def transforma_bin(set_E, set_T, conjunto):
    solucao = range(0, conjunto)
    solucao_bin = np.isin(solucao, set_E)
    return solucao_bin

In [2]:
dados = {}
dados = extract_data()

solucoes_pandas = pd.read_pickle('solucoes.pkl')
solucoes = solucoes_pandas.to_dict()


Iniciando problema sch10.txt
n =  10
Iniciando problema sch20.txt
n =  20
Iniciando problema sch50.txt
n =  50
Iniciando problema sch100.txt
n =  100
Iniciando problema sch200.txt
n =  200
Iniciando problema sch500.txt
n =  500
Iniciando problema sch1000.txt
n =  1000


In [3]:
lista_z = [0.25 , 0.5 , 0.6 , 0.75, 2]
conjunto = 10
problema = 3
h = 0.8

pi = np.array(dados[conjunto][problema]['pi'])
ai = np.array(dados[conjunto][problema]['ai'])
bi = np.array(dados[conjunto][problema]['bi'])

d = int(sum(pi)*h)

for cada_z in lista_z:
    print(cada_z, solucoes[(conjunto,problema,h,cada_z)])

0.25 [ True  True False False False  True  True False  True  True]
0.5 [ True  True False False False  True  True False  True  True]
0.6 [ True  True False False False  True  True False  True  True]
0.75 [ True  True False False False  True  True False  True  True]
2 [ True  True False False False  True  True False  True  True]


In [4]:
#luiza, vou por as funções aqui, e os testes vou manter nas células abaixo
#aí você consegue ver os testes que eu fiz e como eu espero que a função se comporte.

@jit(nopython=True)
def crossover(p1,p2):     #p1,p2 = np.array com soluções, formato booleano.
    tamanho = len(p1)
    metade = int(tamanho/2)
    parte1 = p1[:metade]
    parte2 = p2[metade:]
    filho = np.append(parte1,parte2)
    return filho #falta incluir uma função de reparo de infactíveis




@jit(nopython=True)
def crossover_r(p1,p2):     #p1,p2 = np.array com soluções, formato booleano.
                            #a diferença desse pro anterior é que dois cruzamentos dos mesmos pais
                            #vão retornar filhos diferentes.
                            #a diferença de tempo para 1000 tarefas é de 2us para 20us.
    roleta =  np.array([np.random.random() for i in range(len(p1))])
    filho = np.copy(p1)
    p2 = np.copy(p2)
    for cada_tarefa in np.where(roleta<=0.5): #o filho é igual p1, exceto onde roleta<=0.5.
        filho[cada_tarefa] = p2[cada_tarefa]
        
    return filho #falta incluir uma função de reparo de infactíveis


@jit(nopython=True)
def mutacao(sol,p): #sol = np.array com solução, formato booleano.
                    #p = probabilidade de cada bit sofrer mutação, float ;  [0,1].
    roleta =  np.array([np.random.random() for i in range(len(sol))])
    copia_sol = np.copy(sol)
    for cada_mutante in np.where(roleta<=p):
        copia_sol[cada_mutante] = np.logical_not(copia_sol[cada_mutante])
    return copia_sol




@jit(nopython=True)
def transforma_bin2(set_E, set_T):
    solucao_bin = np.array([True]*(len(set_E)+len(set_T)))
    solucao_bin[set_T] = False
    return solucao_bin

@jit(nopython=True)
def calcula_diversidade(sols,objs):
    melhor = np.argmin(objs)
    len_sols = len(sols[0])
    igualdade =np.array([np.sum(sols[kk]==sols[melhor]) for kk in range(len(sols))]) #conta quantos bits iguais a melhor solucao em cada solução
    #de todos bits da população,                                                                           
    #quantos % são diferentes do melhor?
    #substraí len_sols do numerador e denumerador para desconsiderar quando ele comparara a melhor com ela mesmo
    #mas na prática é irrelevante.
    diversidade_percent = (np.sum(igualdade)-len_sols)/((len(igualdade)*len_sols)-len_sols)
    return diversidade_percent

In [5]:
@jit(nopython=True)
def repara_solucao(set_E,set_T,ai,bi,pi,d):
    sol_ = transforma_bin2(set_E,set_T)
    d_solucao = np.sum(pi[set_E])

    if (d_solucao <= d): #solução não é infactível, vou sair sem fazer nada.
        return (set_E,set_T)

    z = (bi-ai)/(bi+ai)
    violacao = d_solucao-d #o quanto eu preciso reduzir no meu d_solucao para ser factível?
    zsort_set_E = np.flip(np.argsort(z[set_E])) #os primeiros são os maiores Zs no set_E.
                                                #isto é, supostamente os melhores a serem movidos para o set_T.

    pi_cumsum_zsort_set_E = np.cumsum(pi[set_E[zsort_set_E]]) #valor acumulado dos pi na ordem considerada a melhor.
    mudar_ate = np.nonzero(pi_cumsum_zsort_set_E>violacao)[0][0]+1 #qual a primeira tarefa da lista ordenada que excede a violação?
    mudar = set_E[zsort_set_E[:mudar_ate]] #colho os índices dos que precisarão ser alterados
    sol_[mudar] = np.logical_not(sol_[mudar]) #mudo de conjunto itens necessários para reparo
    novo_set_E,novo_set_T = transforma(sol_)
    return novo_set_E,novo_set_T 


@jit(nopython=True)
def oprime_fracos(objs,max_pop):
    len_objs = len(objs) 

    sobreviventes = np.array([True]*len_objs) #começo com todos sobrevivendo.
    oprimir_quantos = len_objs - max_pop

    if oprimir_quantos<=0: #se eu não tiver exemplares demais, não preciso matar ninguém.
        return sobreviventes

    objs_decr = np.flip(np.argsort(objs))

    n_elitismo = max(3,int(max_pop*0.1))
    elite = objs_decr[:n_elitismo]

    roleta =  np.array([np.random.random() for i in range(len(objs))])
    roleta[elite] = 1 

    oprimidos = np.argsort(roleta)[0:oprimir_quantos]

    sobreviventes[oprimidos] = False
    return sobreviventes

In [6]:
a=np.tile(True,4)
b=np.tile(False,4)

crossover(a,b)

array([ True,  True, False, False])

In [7]:
a=np.tile(True,1000)
b=np.tile(False,1000)
%timeit crossover(a,b)

1.13 µs ± 25 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
%timeit crossover_r(a,b)

14.7 µs ± 4.26 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
dd =  mutacao(a,0.3)

14.6 µs ± 5.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
@jit(nopython=True)
def transforma(solucao):
    set_E = np.nonzero(solucao == True)[0] #[0] para retornar o np array e não o tuple.
    set_T = np.nonzero(solucao == False)[0]
    return set_E,set_T


def transforma_bin(set_E, set_T, conjunto):
    solucao = range(0, conjunto)
    solucao_bin = np.isin(solucao, set_E)
    return solucao_bin




# @jit(nopython=True)
def busca_total(solucao_partida, ai, bi, pi, d, lista_tabu): 
    objetivos = {}
    sol_testada = {}
    lista_z = (bi-ai)/(bi+ai)

    for cada_serv in range(0,len(solucao_partida)):
        if cada_serv not in lista_tabu:
            sol_em_teste = solucao_partida.copy()
            sol_em_teste[cada_serv] = not(solucao_partida[cada_serv]) #inverte o serviço de bolso.
            objetivos[cada_serv] = calcula_objetivo(sol_em_teste, ai, bi, pi, d)
            sol_testada[cada_serv] = sol_em_teste

    return objetivos,sol_testada

@jit(nopython=True)
def maximo(a,b):
    if a>b:
        return a
    else:
        return b
    

@jit(nopython=True)
def minimo(a,b):
    if a<b:
        return a
    else:
        return b

#@jit(nopython=True)
def calcula_objetivo(solucao, ai, bi, pi, d):
    
    set_E, set_T = transforma(solucao)
    
    if np.sum(pi[set_E])>d:
        return 99999999999

        
    ai_pi = ai[set_E]/pi[set_E] #apenas do set_E
    bi_pi = bi[set_T]/pi[set_T] #apenas do set_T
    # set_E_pi = pi[set_E]    
    
    # set_T_pi = pi[set_T]
    ai_pi_decr = np.flip(np.argsort(ai_pi)) #ordem de ai_pi e depois por pi.
    bi_pi_decr = np.flip(np.argsort(bi_pi))
    set_E = set_E[ai_pi_decr]
    set_T = set_T[bi_pi_decr]

    objetivo_minimo = calcula_objetivo_minimo(ai, bi, pi, set_E, set_T, d)

    # breakpoint()
    return objetivo_minimo

@jit(nopython=True)
def verifica_factibilidade(solucao, pi, d):
    set_E, set_T = transforma(solucao)
        
    if np.sum(pi[set_E])>d:
        return False
    else:
        return True

@jit(nopython=True)
def calcula_objetivo_minimo(ai, bi, pi, set_E, set_T, d):
    
    sum_pi_final_d = 0
    objetivo_final_d = 0
    objetivo_inicio_0 = 0

    gap_E = d - np.sum(pi[set_E])
    sum_pi_inicio_0 = gap_E
    
    for tarefa in set_E:
        objetivo_final_d += sum_pi_final_d * ai[tarefa]
        objetivo_inicio_0 += sum_pi_inicio_0 * ai[tarefa]
        sum_pi_final_d += pi[tarefa]
        sum_pi_inicio_0 += pi[tarefa]
    
    sum_pi_final_d = 0
    sum_pi_inicio_0 = -gap_E
    for tarefa in set_T:
        objetivo_final_d += (sum_pi_final_d+pi[tarefa])*bi[tarefa]
        objetivo_inicio_0 += (sum_pi_inicio_0+pi[tarefa])*bi[tarefa]
        sum_pi_final_d += pi[tarefa]
        sum_pi_inicio_0 += pi[tarefa]

    if gap_E > pi[set_T[0]]:
        objetivo_inicio_0 = 99999999999

    return minimo(objetivo_final_d,objetivo_inicio_0)

@jit(nopython=True)
def calcula_objetivo_0(ai, bi, pi, set_E, set_T):
    sum_pi = 0
    objetivo = 0

    gap_E = d - np.sum(pi[set_E])
    sum_pi = gap_E

    for tarefa in set_E:
        objetivo += sum_pi * ai[tarefa]
        sum_pi += pi[tarefa]
    
    sum_pi = -gap_E
    for tarefa in set_T:
        objetivo += (sum_pi+pi[tarefa])*bi[tarefa]
        sum_pi += pi[tarefa]
    return objetivo

def busca_local(set_E, set_T, conjunto, ai, bi, pi, d, seq_obj):
    solucao_pre_buscalocal = transforma_bin(set_E, set_T, conjunto)
                
    objs = {}
    sols = {}
    seq_sols = []
    seq_sols.append(solucao_pre_buscalocal)
    obj_original = seq_obj[-1]
    obj_atual = seq_obj[-1]
    continua = True
    contador_aleatorio = 0
    lista_tabu = [conjunto+1]
    contador_tabu = 0
    aceita_pior = False
    contador_pior = 0

    for n in range(1,101): #1º critério de parada - número máximo de vizinhanças                    
        obj_ant = obj_atual
        if n == 1:
            objs,sols = busca_total(solucao_pre_buscalocal, ai, bi, pi, d, lista_tabu)
        else:
            objs,sols = busca_total(sol_atual, ai, bi, pi, d, lista_tabu)

        objs = dict(sorted(objs.items(), key=lambda item: item[1], reverse=False))
        if len(objs) == 0:
            break
        obj_atual = objs[next(iter(objs))]
        sol_atual = sols[next(iter(objs))]
        if obj_atual >= obj_ant:
            if aceita_pior == False:
                aceita_pior = True
                contador_pior = 1
            else:
                if contador_pior >= maximo(conjunto*0.1, 10): #2º critério de parada - número máximo de soluções piores aceitas  
                    break
                else:
                    contador_pior += 1
        elif obj_atual < seq_obj[-1]:
            seq_obj.append(obj_atual)
            seq_sols.append(sol_atual)
            contador_pior = 0
    return seq_obj, seq_sols


In [11]:
sol= np.tile(True,conjunto)
set_E,set_T = transforma(sol)

In [12]:
%timeit repara_solucao(set_E,set_T,ai,bi,pi,d)

The slowest run took 9.09 times longer than the fastest. This could mean that an intermediate result is being cached.
11.1 µs ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
set_E,set_T  = repara_solucao(set_E,set_T,ai,bi,pi,d)
a = transforma_bin2(set_E,set_T)
calcula_objetivo(a,ai,bi,pi,d)

1828.0

In [14]:
objs = np.arange(0,10)
max_pop = 5
%timeit oprime_fracos(objs,max_pop)

The slowest run took 6.31 times longer than the fastest. This could mean that an intermediate result is being cached.
6.33 µs ± 6.45 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
oprime_fracos(objs,max_pop)

array([ True, False, False, False, False,  True, False,  True,  True,
        True])

In [16]:
sols = [np.array(list(np.binary_repr(kk,10)),dtype=int) for kk in range(800,1023)]
sols = np.array(sols,dtype=bool)
sols #gerei uma população

array([[ True,  True, False, ..., False, False, False],
       [ True,  True, False, ..., False, False,  True],
       [ True,  True, False, ..., False,  True, False],
       ...,
       [ True,  True,  True, ...,  True, False, False],
       [ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True,  True, False]])

In [17]:
objs = np.array([calcula_objetivo(cada_sol,ai,bi,pi,d) for cada_sol in sols]) #calculei os objetivos

In [18]:
objs

array([1.515e+03, 1.329e+03, 1.268e+03, 1.166e+03, 1.597e+03, 1.471e+03,
       1.440e+03, 1.398e+03, 1.044e+03, 9.360e+02, 9.140e+02, 8.900e+02,
       1.224e+03, 1.176e+03, 1.184e+03, 1.220e+03, 1.277e+03, 1.163e+03,
       1.138e+03, 1.108e+03, 1.459e+03, 1.405e+03, 1.410e+03, 1.440e+03,
       9.360e+02, 9.000e+02, 9.140e+02, 9.620e+02, 1.216e+03, 1.240e+03,
       1.284e+03, 1.392e+03, 1.608e+03, 1.386e+03, 1.344e+03, 1.206e+03,
       1.720e+03, 1.558e+03, 1.546e+03, 1.468e+03, 1.126e+03, 9.820e+02,
       9.790e+02, 9.190e+02, 1.336e+03, 1.252e+03, 1.279e+03, 1.279e+03,
       1.356e+03, 1.206e+03, 1.200e+03, 1.134e+03, 1.568e+03, 1.478e+03,
       1.502e+03, 1.496e+03, 1.004e+03, 9.320e+02, 9.650e+02, 9.770e+02,
       1.314e+03, 1.302e+03, 1.000e+11, 1.000e+11, 1.503e+03, 1.359e+03,
       1.319e+03, 1.259e+03, 1.725e+03, 1.641e+03, 1.631e+03, 1.631e+03,
       1.130e+03, 1.064e+03, 1.063e+03, 1.081e+03, 1.450e+03, 1.444e+03,
       1.000e+11, 1.000e+11, 1.335e+03, 1.263e+03, 

In [19]:
@jit(nopython=True)
def calcula_diversidade(sols,objs):
    melhor = np.argmin(objs)
    len_sols = len(sols[0])
    igualdade =np.array([np.sum(sols[kk]==sols[melhor]) for kk in range(len(sols))]) #conta quantos bits iguais a melhor solucao em cada solução
    #de todos bits da população,                                                                           
    #quantos % são diferentes do melhor?
    #substraí len_sols do numerador e denumerador para desconsiderar quando ele comparara a melhor com ela mesmo
    #mas na prática é irrelevante.
    diversidade_percent = (np.sum(igualdade)-len_sols)/((len(igualdade)*len_sols)-len_sols)
    return diversidade_percent

In [20]:
%timeit calcula_diversidade(sols,objs)

32.5 µs ± 4.61 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
np.sum(pi[set_E])

97

In [22]:
d
    

100

In [23]:
set_E,set_T = repara_solucao(set_E,set_T,ai,bi,pi,d)

In [24]:
np.sum(pi[set_E])

97

In [25]:
sol = np.array([True,  True,  True,  True,  True, False,  True,  True,  True,  True])
set_E,set_T = transforma(sol)
set_E,set_T




(array([0, 1, 2, 3, 4, 6, 7, 8, 9], dtype=int64), array([5], dtype=int64))

In [26]:
repara_solucao(set_E,set_T,ai,bi,pi,d)

(array([0, 1, 2, 3, 4, 7, 8, 9], dtype=int64), array([5, 6], dtype=int64))

NameError: name 'mudar' is not defined